### Making follower networks from Bluesky

We are going to use Bluesky to create an ego network of your account. We will get all of the followers and following of your account, and then we will get the followers and following of those accounts. This will create a network of your account and the accounts that are connected to it.

We will then visualize the network using `ggraph`.

In [ ]:
from atproto import Client
import config
import json
import time


def get_all_follows(client, handle):
    """
    Get all of the users that the given handle follows.
    """
    following = []
    next_cursor = None
    while True:
        response = client.get_follows(handle, limit=100, cursor=next_cursor)
        following.extend(response.follows)
        if not response.cursor:
            break
        next_cursor = response.cursor
    return following

def make_network_from_following(following_dict):
    """
    Take a dictionary of users and their following lists and create a graph.
    The graph is a list of tuples, where each tuple is (user, followed_user).
    """
    following_graph = []
    for user, following in following_dict.items():
        for follow in following:
            if follow in following_dict:
                following_graph.append((user, follow))
    return following_graph


def main(username) -> None:
    # Create a client and login
    at_client = Client()
    at_client.login(config.username, config.password)
    # Get the followers and following of the username
    # Get all of the accounts the user follows
    following = get_all_follows(at_client, user_to_analyze)
    # For each of those accounts, get their followers
    following_dict = {}
    for user in following:
        curr_user_following = get_all_follows(at_client, user['handle'])
        following_dict[user['handle']] = set([x['handle'] for x in curr_user_following])
        # Sleep for a second to be a good citizen
        time.sleep(1)
    # Create a graph from the following dictionary
    following_graph = make_network_from_following(following_dict)
    with open('following_graph.csv', 'w') as f:
        f.write('from,to\n')
        for user, following in following_graph:
            f.write(f'{user},{following}\n')

In [ ]:
main('jeremydfoote.com')